In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score


In [2]:
#loading the dataset
salary_train1=pd.read_csv("C:\\Users\\Lenovo\\Downloads\\SalaryData_Train.csv")
salary_test=pd.read_csv("C:\\Users\\Lenovo\\Downloads\\SalaryData_Test.csv")
salary_train1

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30159,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
#education no not given in problem statement, hence dropping
salary_train=salary_train1.drop(['educationno'],axis=1)

In [4]:
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   maritalstatus  30161 non-null  object
 4   occupation     30161 non-null  object
 5   relationship   30161 non-null  object
 6   race           30161 non-null  object
 7   sex            30161 non-null  object
 8   capitalgain    30161 non-null  int64 
 9   capitalloss    30161 non-null  int64 
 10  hoursperweek   30161 non-null  int64 
 11  native         30161 non-null  object
 12  Salary         30161 non-null  object
dtypes: int64(4), object(9)
memory usage: 3.0+ MB


In [5]:
#label encoding categorical data
le=preprocessing.LabelEncoder()
salary_train['workclass']=le.fit_transform(salary_train['workclass'])
salary_train['education']=le.fit_transform(salary_train['education'])
salary_train['maritalstatus']=le.fit_transform(salary_train['maritalstatus'])
salary_train['occupation']=le.fit_transform(salary_train['occupation'])
salary_train['relationship']=le.fit_transform(salary_train['relationship'])
salary_train['race']=le.fit_transform(salary_train['race'])
salary_train['sex']=le.fit_transform(salary_train['sex'])
salary_train['native']=le.fit_transform(salary_train['native'])
salary_train['Salary']=le.fit_transform(salary_train['Salary'])

In [6]:
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            30161 non-null  int64
 1   workclass      30161 non-null  int32
 2   education      30161 non-null  int32
 3   maritalstatus  30161 non-null  int32
 4   occupation     30161 non-null  int32
 5   relationship   30161 non-null  int32
 6   race           30161 non-null  int32
 7   sex            30161 non-null  int32
 8   capitalgain    30161 non-null  int64
 9   capitalloss    30161 non-null  int64
 10  hoursperweek   30161 non-null  int64
 11  native         30161 non-null  int32
 12  Salary         30161 non-null  int32
dtypes: int32(9), int64(4)
memory usage: 2.0 MB


In [7]:
#normalizing numerical data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(salary_train.loc[:,('age','capitalgain','capitalloss','hoursperweek')])
salary_train.loc[:,('age','capitalgain','capitalloss','hoursperweek')]=scaler.transform(salary_train.loc[:,('age','capitalgain','capitalloss','hoursperweek')])
salary_train

,age,workclass,education,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,0.301370,5,9,4,0,1,4,1,0.021740,0.0,0.397959,37,0
1,0.452055,4,9,2,3,0,4,1,0.000000,0.0,0.122449,37,0
2,0.287671,2,11,0,5,1,4,1,0.000000,0.0,0.397959,37,0
3,0.493151,2,1,2,5,0,2,1,0.000000,0.0,0.397959,37,0
4,0.150685,2,9,2,9,5,2,0,0.000000,0.0,0.397959,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,0.136986,2,7,2,12,5,4,0,0.000000,0.0,0.377551,37,0
30157,0.315068,2,11,2,6,0,4,1,0.000000,0.0,0.397959,37,1
30158,0.561644,2,11,6,0,4,4,0,0.000000,0.0,0.397959,37,0
30159,0.068493,2,11,4,0,3,4,1,0.000000,0.0,0.193878,37,0


In [8]:
#defining train data
from sklearn.model_selection import train_test_split, cross_val_score
x=salary_train.iloc[:,:-1]
y=salary_train.loc[:,'Salary']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5)

# Grid search cv

In [9]:
clf = SVC()
param_grid = [{'kernel':['rbf'],'gamma':[10,5,1,0.5],'C':[10,0.1,0.001] }]
gsv = GridSearchCV(clf,param_grid,cv=10)
gsv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [10, 0.1, 0.001], 'gamma': [10, 5, 1, 0.5],
                          'kernel': ['rbf']}])

In [10]:
gsv.best_params_ , gsv.best_score_ 

({'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}, 0.8169098143236073)

In [11]:
#building model with grid search results
clf = SVC(C= 1, gamma = 10)
clf.fit(x_train , y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(y_test, y_pred)

Accuracy = 81.10204893574696


array([[10606,   732],
       [ 2118,  1625]], dtype=int64)